In [1]:
import pandas as pd

In [2]:
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf 
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType,TimestampType
from pyspark.sql.functions import col,current_timestamp,to_date,hour,dayofweek
spark = SparkSession \
    .builder \
    .master('local[*]') \
    .config("spark.driver.memory", "14g") \
    .appName('my-cool-app') \
    .getOrCreate()

sc=spark.sparkContext

23/02/10 01:50:37 WARN Utils: Your hostname, abdi-VivoBook-ASUSLaptop-X415EP-R465EP resolves to a loopback address: 127.0.1.1; using 172.27.172.43 instead (on interface wlo1)
23/02/10 01:50:37 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/10 01:50:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
schema = StructType([ \
        StructField("DEVICE_CODE", StringType(), True), 
        StructField("SYSTEM_ID",IntegerType(),True), \
        StructField("ORIGINE_CAR_KEY",StringType(),True), \
        StructField("FINAL_CAR_KEY",StringType(),True), \
        StructField("CHECK_STATUS_KEY", IntegerType(), True), \
        StructField("COMPANY_ID", StringType(), True), \
        StructField("PASS_DAY_TIME", TimestampType(), True)
    ])

In [4]:
df=spark.read.csv('SampleData.csv',header=True,schema=schema)
df.show(1)

23/02/10 01:50:42 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 9, schema size: 7
CSV file: file:///home/abdi/BigData/Project/SampleData_Wednesday.csv/part-00000-d369a0f2-087a-4e02-b4b8-c5f2b9563970-c000.csv
+-----------+---------+---------------+-------------+----------------+----------+-------------------+
|DEVICE_CODE|SYSTEM_ID|ORIGINE_CAR_KEY|FINAL_CAR_KEY|CHECK_STATUS_KEY|COMPANY_ID|      PASS_DAY_TIME|
+-----------+---------+---------------+-------------+----------------+----------+-------------------+
|     900174|      283|       91843693|     91843693|               5|       264|2021-12-29 07:15:35|
+-----------+---------+---------------+-------------+----------------+----------+-------------------+
only showing top 1 row



In [5]:
Traffic = df.rdd.map(lambda x: ((x['FINAL_CAR_KEY'] , x['PASS_DAY_TIME'].date()), x['DEVICE_CODE']))


In [6]:
#this is just preproccessing data and making key_vals and groupby keys
Traffic = Traffic.groupByKey()
Traffic = Traffic.map(lambda x : (x[0],sorted(set(list(x[1])))))

In [7]:
from operator import add
# we need just camera part so we are deleting which car they are belong to
Traffic = Traffic.map(lambda x : tuple(x[1]))



In [8]:
import itertools
# in this function we are combining all c_frequents from (c-1)_frequent items
#complete description is in report file
def make_next_item_sub(x , num):
    new = []
    for i in range(len(x) - 1):
        for j in range(i+1 ,len(x)):
            maker = []
            if type(x[i]) is tuple:
                for t in x[i]:
                    maker.append(t)
            else :
                 maker.append(x[i])
            if type(x[j]) is tuple:
                for t in x[j]:
                    maker.append(t)
            else :
                 maker.append(x[j])
            new_set = set(maker)
            temp = list(new_set)
            temp.sort()
            temp = tuple(temp)
            if len(temp) == num:
                new.append(tuple(temp))
            
    new = set(new)
    new = list(new)
    return new

#in this function we are deleting items that are not frequent
def make_basket_smaller(x , distinct):
    new = []
    for item in x:
        if item in distinct:
            new.append(item)
    return tuple(new)

def prune_unwanted(x , c):
    new = []
    for item in x:
        if len(item ) == c:
            new.append(item)
    return new

In [9]:
import itertools
#this is just impleenting of A-priori algorithm
def A_priori(rdd , support , c , max_iter):
    frequents = []
    while (c <= max_iter):
        rdd = rdd.map(lambda x : make_next_item_sub(x,c))
        rdd = rdd.map(lambda x: prune_unwanted(x,c))
        temp = rdd.flatMap(lambda x : x)
        help = temp.map(lambda x : (x,1)).reduceByKey(add).filter(lambda x : x[1]>= support)
        frequents.append(help.collect())
        distinct_val = help.map(lambda x : x[0]).collect()
        rdd = rdd.map(lambda x : make_basket_smaller(x,distinct_val))
        c += 1
    return frequents

In [10]:
#I set 4000 for one_freq_items
#because I have checked and a number between 3500 - 4500 is good for filtering one_freq_item and this is reducing 40 percent of data
one_item = Traffic.flatMap(lambda x : x).map(lambda x: (x,1)).reduceByKey(add).filter(lambda x: x[1] > 4000)
one_freq_item = one_item.collect()

23/02/10 01:50:43 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 9, schema size: 7
CSV file: file:///home/abdi/BigData/Project/SampleData_Wednesday.csv/part-00000-d369a0f2-087a-4e02-b4b8-c5f2b9563970-c000.csv


In [32]:
one_freq_item

[('900135', 14079),
 ('900174', 13844),
 ('203902', 17189),
 ('100701090', 8225),
 ('230201', 5362),
 ('119', 17103),
 ('900213', 15885),
 ('22009923', 5960),
 ('900272', 11655),
 ('22009972', 12898),
 ('900167', 21775),
 ('100701260', 11225),
 ('900247', 9258),
 ('112', 5535),
 ('900162', 5455),
 ('900156', 9289),
 ('115', 7193),
 ('900179', 5043),
 ('900170', 6415),
 ('631346', 7889),
 ('205202', 6499),
 ('631830', 10228),
 ('100701096', 8639),
 ('631362', 7226),
 ('22010072', 4477),
 ('631366', 5077),
 ('100700804', 35214),
 ('900164', 39451),
 ('900202', 34718),
 ('900222', 60930),
 ('100700847', 8353),
 ('100701119', 11897),
 ('900236', 43772),
 ('631361', 12296),
 ('900228', 11188),
 ('900248', 10769),
 ('900244', 78692),
 ('900233', 22291),
 ('631354', 7897),
 ('900194', 4783),
 ('103001', 9664),
 ('103002', 13567),
 ('155', 7696),
 ('900149', 21450),
 ('900113', 4301),
 ('900277', 12812),
 ('900232', 7073),
 ('168', 5121),
 ('100700894', 7917),
 ('631757', 10307),
 ('900151', 9

In [11]:
one_item = one_item.map(lambda x : x[0]).collect()
Traffic = Traffic.map(lambda x : make_basket_smaller (x , one_item))


In [12]:
#making two_sub from one_frequents
two_sub = Traffic.map(lambda x:make_next_item_sub(x,2)).flatMap(lambda x : x).map(lambda x : (x,1)).reduceByKey(add).filter(lambda x : x[1] > 1200)
two_freq_item = two_sub.collect()

In [31]:
list_of_two_sub = two_sub.map(lambda x: x[0]).collect()
two_freq_item

[(('100701130', '900102'), 5071),
 (('100700841', '900221'), 1312),
 (('100700965', '900150'), 1830),
 (('900141', '900228'), 1453),
 (('100700853', '631634'), 8520),
 (('631633', '900244'), 20751),
 (('631633', '900222'), 6244),
 (('22010118', '900215'), 3336),
 (('900215', '900256'), 6448),
 (('100700871', '900235'), 5482),
 (('900142', '900213'), 2270),
 (('900213', '900264'), 3617),
 (('22009971', '22009972'), 4537),
 (('631368', '900234'), 1358),
 (('100700864', '900218'), 3420),
 (('631634', '900243'), 1705),
 (('230204', '900247'), 1859),
 (('100701130', '900182'), 1273),
 (('900155', '900272'), 1669),
 (('900185', '900242'), 4631),
 (('900152', '900208'), 1580),
 (('631357', '900246'), 8452),
 (('100700841', '900268'), 5142),
 (('631633', '900202'), 5029),
 (('900101', '900259'), 4400),
 (('900212', '900259'), 4459),
 (('100700841', '900259'), 6429),
 (('900101', '900268'), 8023),
 (('900212', '900268'), 3977),
 (('900193', '900212'), 3097),
 (('100700853', '900245'), 3357),
 (

In [14]:
#filtering baskets to hold just two_freqs
two_freq = Traffic.map(lambda x : make_next_item_sub(x,2)).map(lambda x : make_basket_smaller(x , list_of_two_sub))
two_freq.collect()

[(),
 (('100700804', '900215'), ('100700804', '900164'), ('100700820', '900164')),
 (('900202', '900222'),),
 (),
 (),
 (('900268', '900269'), ('900266', '900269')),
 (),
 (('100700841', '900102'),
  ('231', '900236'),
  ('100701130', '631357'),
  ('231', '631357'),
  ('100701130', '900102'),
  ('631357', '900102'),
  ('631357', '900236'),
  ('900102', '900236'),
  ('100700841', '900236'),
  ('100700841', '631357'),
  ('100700841', '231')),
 (('900155', '900269'), ('631361', '900155'), ('631361', '900269')),
 (('100700841', '900101'),),
 (('100700841', '101301'),
  ('212501', '900221'),
  ('100700841', '212501'),
  ('101301', '900236'),
  ('100700841', '900236'),
  ('100700841', '900101'),
  ('900221', '900236'),
  ('212501', '900236'),
  ('900101', '900236'),
  ('101301', '900101'),
  ('100700841', '900221')),
 (('100700804', '900174'),),
 (),
 (('100700965', '900150'),),
 (),
 (('900141', '900228'),),
 (('231', '900236'),
  ('231', '631357'),
  ('631357', '900236'),
  ('100700841', '

In [15]:
#making three_sub from two_sub frequents
three_sub = two_freq.map(lambda x : make_next_item_sub(x,3))
rr = three_sub.flatMap(lambda x: x).map(lambda x : (x,1)).reduceByKey(add).filter(lambda x : x[1]>= 1200)
distinct_list = rr.map(lambda x :  x[0]).collect()
three_sub = three_sub.map(lambda x :make_basket_smaller(x , distinct_list))
three_freq_item = rr.collect()
three_freq_item

[(('22010118', '900215', '900256'), 2153),
 (('103002', '22009971', '900218'), 1203),
 (('175', '900191', '900218'), 5038),
 (('100700853', '900142', '900244'), 6937),
 (('631633', '900202', '900212'), 1979),
 (('100700841', '900101', '900259'), 2504),
 (('900202', '900212', '900244'), 3769),
 (('100700841', '900101', '900268'), 2725),
 (('100700853', '631633', '631634'), 2566),
 (('100700853', '631634', '900212'), 3472),
 (('900212', '900222', '900244'), 4331),
 (('900225', '900243', '900269'), 1699),
 (('100700853', '900101', '900142'), 1252),
 (('231', '900236', '900255'), 2886),
 (('900212', '900236', '900244'), 4534),
 (('119', '123', '900266'), 1355),
 (('100700853', '900152', '900212'), 2865),
 (('900102', '900142', '900212'), 2264),
 (('900142', '900244', '900269'), 1624),
 (('631634', '900142', '900212'), 3108),
 (('212802', '900215', '900256'), 1276),
 (('631357', '631795', '900246'), 1209),
 (('205201', '205202', '22009971'), 1211),
 (('100700853', '900102', '900212'), 1673)

In [16]:
#making four_subs from three frequent sub
four_sub = three_sub.map(lambda x : make_next_item_sub(x , 4))
rr =four_sub.flatMap(lambda x: x).map(lambda x : (x,1)).reduceByKey(add).filter(lambda x : x[1]>= 1200)
distinct_list = rr.map(lambda x :  x[0]).collect()
four_sub = four_sub.map(lambda x :make_basket_smaller(x , distinct_list))
four_freq_item = rr.collect()

In [17]:
#there is no five_sub even with one repeat in baskets
five_sub = four_sub.map(lambda x : make_next_item_sub(x , 5))
rr =five_sub.flatMap(lambda x: x).map(lambda x : (x,1)).reduceByKey(add).filter(lambda x : x[1]>= 1)
distinct_list = rr.map(lambda x :  x[0]).collect()
five_sub = five_sub.map(lambda x :make_basket_smaller(x , distinct_list))
rr.collect()

[]

In [18]:
four_sub_list = rr

In [19]:
dict_one = {}
for item in one_freq_item:
    dict_one[item[0]] = item[1]

dict_two = {}
for item in two_freq_item:
    dict_two[item[0]] = item[1]

dict_three = {}
for item in three_freq_item:
    dict_three[item[0]] = item[1]

dict_four = {}
for item in four_freq_item:
    dict_four[item[0]] = item[1]

In [20]:
#making A -> B Assosiation rule
#setting confidence score to 0.6
one_to_one_conf = []
for item in dict_two:
    first = item[0]
    second = item[1]
    num_first = dict_one[first]
    num_sec = dict_one[second]
    first_conf = dict_two[item]/num_first
    second_conf = dict_two[item]/num_sec
    if first_conf > 0.6:
        one_to_one_conf.append(((first , second) , first_conf))
    if second_conf > 0.6:
        one_to_one_conf.append(((second , first) , second_conf))

one_to_one_conf = sorted(one_to_one_conf , key = lambda x: -x[1])
for item in one_to_one_conf:
    print('{} -> {} with confidence score of : {}'.format(item[0][0] , item[0][1] , item[1]))

100700834 -> 100700804 with confidence score of : 0.8514253502979546
631833 -> 900269 with confidence score of : 0.8215403882279274
123 -> 900266 with confidence score of : 0.776657060518732
631356 -> 900269 with confidence score of : 0.6831453730882125
22010094 -> 22010088 with confidence score of : 0.6631683168316832
900244 -> 900212 with confidence score of : 0.6619478473034107
900212 -> 900244 with confidence score of : 0.6541668759732757
175 -> 900191 with confidence score of : 0.6387857142857143
22010043 -> 22010044 with confidence score of : 0.6098388464800678


In [21]:
#making A,B -> C association rules
#setting confidence score to 0.65
two_to_one_conf = []
for item in dict_three:
    first = item[0]
    second = item[1]
    third = item[2]
    first_tup = [first , second]
    first_tup.sort()
    first_tup = tuple(first_tup)
    second_tup = [second , third]
    second_tup.sort()
    second_tup = tuple(second_tup)
    third_tup = [first , third]
    third_tup.sort()
    third_tup = tuple(third_tup)
    first_conf = dict_three[item] / dict_two[first_tup]
    second_conf = dict_three[item]/ dict_two[second_tup]
    third_conf = dict_three[item] / dict_two[third_tup]
    if first_conf > 0.65:
        two_to_one_conf.append(((first_tup , third) , first_conf))
    if second_conf > 0.65:
        two_to_one_conf.append(((second_tup , first) , second_conf))
    if third_conf > 0.65:
        two_to_one_conf.append(((third_tup , second) , third_conf))
two_to_one_conf = sorted(two_to_one_conf , key = lambda x: -x[1])
for item in two_to_one_conf:
        print('{} -> {} with confidence score of : {}'.format(item[0][0] , item[0][1] , item[1]))

('100700834', '900235') -> 100700804 with confidence score of : 0.9353667392883079
('900213', '900244') -> 900212 with confidence score of : 0.9215394780310538
('631356', '900246') -> 900269 with confidence score of : 0.9150159744408946
('900227', '900259') -> 900269 with confidence score of : 0.9121140142517815
('100700841', '202601') -> 900236 with confidence score of : 0.9032418952618454
('22009977', '900225') -> 900268 with confidence score of : 0.8985507246376812
('100700804', '123') -> 900266 with confidence score of : 0.896
('100700841', '231') -> 900236 with confidence score of : 0.8919226393629124
('631633', '900213') -> 900212 with confidence score of : 0.8813490451036164
('100700839', '900212') -> 900244 with confidence score of : 0.8774556780067082
('900101', '900244') -> 900212 with confidence score of : 0.8738738738738738
('900212', '900264') -> 900213 with confidence score of : 0.8674373795761079
('900142', '900244') -> 900212 with confidence score of : 0.865350089766606

In [22]:
#making A,B,C -> D association rules
#setting confidence score to 0.65
three_to_one_conf = []
for item in dict_four:
    first = item[0]
    second = item[1]
    third = item[2]
    fourth = item[3]
    first_tup = [first , second, third]
    first_tup.sort()
    first_tup = tuple(first_tup)
    second_tup = [first , second , fourth]
    second_tup.sort()
    second_tup = tuple(second_tup)
    third_tup = [first , third , fourth]
    third_tup.sort()
    third_tup = tuple(third_tup)
    fourth_tup = [second, third , fourth]
    fourth_tup.sort()
    fourth_tup = tuple(fourth_tup)
    first_conf = dict_four[item] / dict_three[first_tup]
    second_conf = dict_four[item]/ dict_three[second_tup]
    third_conf = dict_four[item] / dict_three[third_tup]
    fourth_conf = dict_four[item] / dict_three[fourth_tup]
    if first_conf > 0.65:
        three_to_one_conf.append(((first_tup , fourth) , first_conf))
    if second_conf > 0.65:
        three_to_one_conf.append(((second_tup , third) , second_conf))
    if third_conf > 0.65:
        three_to_one_conf.append(((third_tup , second) , third_conf))
    if fourth_conf > 0.65:
        three_to_one_conf.append(((third_tup , first) , fourth_conf))
three_to_one_conf = sorted(three_to_one_conf , key = lambda x: -x[1])
for item in three_to_one_conf:
        print('{} -> {} with confidence score of : {}'.format(item[0][0] , item[0][1] , item[1]))

('900212', '900244', '900264') -> 900212 with confidence score of : 0.9529914529914529
('631633', '900213', '900244') -> 900212 with confidence score of : 0.9314115308151093
('100700841', '231', '900255') -> 900236 with confidence score of : 0.9296210363495746
('631634', '900213', '900244') -> 900212 with confidence score of : 0.9212656364974245
('100700839', '900142', '900212') -> 900244 with confidence score of : 0.897687181497452
('100700839', '100700853', '900212') -> 900244 with confidence score of : 0.8975460122699387
('631633', '900152', '900212') -> 900244 with confidence score of : 0.8880760790051208
('900212', '900244', '900264') -> 900213 with confidence score of : 0.8869318181818182
('631633', '900202', '900212') -> 900244 with confidence score of : 0.886811520970187
('631633', '900212', '900269') -> 900244 with confidence score of : 0.8834586466165414
('631633', '900101', '900244') -> 900212 with confidence score of : 0.8827852998065764
('100700853', '631633', '900212') ->

#######<br/>
#######<br/>
SON Algorithm<br>
#######<br>
#######<br>

In [23]:
#we are dividing our dataframe into 3 rdd to implement SON algorithm
Traffic = Traffic.zipWithIndex()
rdd1 = Traffic.filter(lambda x : x[1] % 3 == 0).map(lambda x : x[0])
rdd2 = Traffic.filter(lambda x : x[1] % 3 == 1).map(lambda x : x[0])
rdd3 = Traffic.filter(lambda x: x[1] % 3 == 2).map(lambda x : x[0])

In [24]:
one_sub_rdd1 = rdd1
one_sub_rdd2 = rdd2
one_sub_rdd3 = rdd3

In [25]:
two_sub_rdd1 = one_sub_rdd1.map(lambda x : make_next_item_sub(x,2))
rr = two_sub_rdd1.flatMap(lambda x: x).map(lambda x : (x,1)).reduceByKey(add).filter(lambda x : x[1]>= 350)
distinct_list = rr.map(lambda x :  x[0]).collect()
two_sub_rdd1 = two_sub_rdd1.map(lambda x :make_basket_smaller(x , distinct_list))
rr.collect()

[(('900141', '900228'), 510),
 (('631633', '900244'), 7167),
 (('631633', '900222'), 2126),
 (('100700965', '900150'), 613),
 (('100700871', '900235'), 1842),
 (('631368', '900234'), 432),
 (('631634', '900243'), 594),
 (('230204', '900247'), 627),
 (('900101', '900207'), 533),
 (('900101', '900225'), 1072),
 (('900207', '900212'), 1280),
 (('900212', '900225'), 1469),
 (('900193', '900212'), 1046),
 (('631363', '900214'), 371),
 (('631363', '900269'), 860),
 (('100701130', '900102'), 1759),
 (('631357', '900225'), 569),
 (('100701130', '900269'), 845),
 (('631363', '900255'), 376),
 (('100700841', '900246'), 851),
 (('100700841', '900221'), 438),
 (('631368', '900266'), 387),
 (('100700864', '101301'), 763),
 (('631633', '900164'), 644),
 (('22010118', '900215'), 1130),
 (('900215', '900256'), 2161),
 (('100701130', '900182'), 436),
 (('631357', '900246'), 2887),
 (('100700881', '900138'), 449),
 (('100700857', '900164'), 959),
 (('631633', '900236'), 884),
 (('900223', '900246'), 597

In [26]:
three_sub_rdd1 = two_sub_rdd1.map(lambda x : make_next_item_sub(x,3))
rr1 = three_sub_rdd1.flatMap(lambda x: x).map(lambda x : (x,1)).reduceByKey(add).filter(lambda x : x[1]>= 350)
distinct_list = rr1.map(lambda x :  x[0]).collect()
three_sub_rdd1_temp = three_sub_rdd1.map(lambda x :make_basket_smaller(x , distinct_list))
rr1.collect()

[(('175', '900191', '900218'), 1720),
 (('900101', '900212', '900225'), 372),
 (('900212', '900222', '900244'), 1441),
 (('900225', '900243', '900269'), 571),
 (('22010118', '900215', '900256'), 754),
 (('900212', '900236', '900244'), 1538),
 (('100700839', '631634', '900207'), 363),
 (('100700841', '900101', '900268'), 894),
 (('100700841', '900101', '900259'), 850),
 (('900102', '900142', '900212'), 793),
 (('100700853', '900142', '900244'), 2392),
 (('900142', '900244', '900269'), 572),
 (('100700853', '900142', '900202'), 379),
 (('631634', '900142', '900212'), 1067),
 (('100700853', '631634', '900212'), 1222),
 (('900202', '900212', '900244'), 1252),
 (('631357', '631795', '900246'), 396),
 (('231', '900236', '900255'), 959),
 (('100700841', '631357', '900246'), 353),
 (('100700853', '900102', '900212'), 569),
 (('100700853', '900152', '900212'), 944),
 (('100700804', '100700834', '900222'), 404),
 (('100700853', '631633', '631634'), 904),
 (('100', '631634', '900207'), 394),
 (('

In [27]:
#making two_sub_freq from one_sub and similarly three_sub _from two_sub
two_sub_rdd2 = one_sub_rdd2.map(lambda x : make_next_item_sub(x,2))
rr = two_sub_rdd2.flatMap(lambda x: x).map(lambda x : (x,1)).reduceByKey(add).filter(lambda x : x[1]>= 350)
distinct_list = rr.map(lambda x :  x[0]).collect()
two_sub_rdd2= two_sub_rdd2.map(lambda x :make_basket_smaller(x , distinct_list))

three_sub_rdd2 = two_sub_rdd2.map(lambda x : make_next_item_sub(x,3))
rr2 = three_sub_rdd2.flatMap(lambda x: x).map(lambda x : (x,1)).reduceByKey(add).filter(lambda x : x[1]>= 350)
distinct_list = rr2.map(lambda x :  x[0]).collect()
three_sub_rdd2_temp = three_sub_rdd2.map(lambda x :make_basket_smaller(x , distinct_list))
rr2.collect()

[(('22010118', '900215', '900256'), 748),
 (('103002', '22009971', '900218'), 368),
 (('631633', '900202', '900212'), 674),
 (('100700841', '900101', '900259'), 838),
 (('900202', '900212', '900244'), 1263),
 (('100700841', '900101', '900268'), 931),
 (('100700804', '100700834', '900222'), 399),
 (('119', '123', '900266'), 466),
 (('100700853', '900142', '900244'), 2282),
 (('100700853', '900152', '900212'), 958),
 (('100700804', '100700834', '900202'), 371),
 (('100700839', '631634', '900207'), 368),
 (('205201', '205202', '22009971'), 386),
 (('175', '900191', '900218'), 1697),
 (('900225', '900246', '900268'), 373),
 (('900212', '900236', '900244'), 1513),
 (('900164', '900212', '900244'), 449),
 (('212802', '22010118', '900215'), 372),
 (('212802', '900215', '900256'), 430),
 (('100700853', '631633', '631634'), 820),
 (('100700853', '631634', '900212'), 1098),
 (('900212', '900222', '900244'), 1412),
 (('231', '900236', '900255'), 958),
 (('100700853', '231', '900236'), 350),
 (('9

In [28]:
#just same algorithm that explained before
two_sub_rdd3 = one_sub_rdd3.map(lambda x : make_next_item_sub(x,2))
rr = two_sub_rdd3.flatMap(lambda x: x).map(lambda x : (x,1)).reduceByKey(add).filter(lambda x : x[1]>= 350)
distinct_list = rr.map(lambda x : x[0]).collect()
two_sub_rdd3= two_sub_rdd3.map(lambda x :make_basket_smaller(x , distinct_list))

three_sub_rdd3 = two_sub_rdd3.map(lambda x : make_next_item_sub(x,3))
rr3 = three_sub_rdd3.flatMap(lambda x: x).map(lambda x : (x,1)).reduceByKey(add).filter(lambda x : x[1]>= 350)
distinct_list = rr3.map(lambda x :  x[0]).collect()
three_sub_rdd3_temp = three_sub_rdd3.map(lambda x :make_basket_smaller(x , distinct_list))
rr3.collect()

[(('103002', '22009971', '900218'), 410),
 (('100700853', '900142', '900244'), 2263),
 (('100700853', '631633', '631634'), 842),
 (('100700853', '631634', '900212'), 1152),
 (('900212', '900222', '900244'), 1478),
 (('900101', '900212', '900268'), 361),
 (('100700804', '100700834', '900222'), 374),
 (('100700853', '900101', '900142'), 436),
 (('900202', '900212', '900244'), 1254),
 (('100700841', '900101', '900268'), 900),
 (('100700841', '900101', '900259'), 816),
 (('231', '900236', '900255'), 969),
 (('212802', '22010118', '900215'), 366),
 (('100700853', '900142', '900202'), 375),
 (('212802', '900215', '900256'), 434),
 (('175', '900191', '900218'), 1621),
 (('900212', '900236', '900244'), 1483),
 (('100700853', '900152', '900212'), 963),
 (('900225', '900259', '900268'), 941),
 (('205201', '205202', '22009971'), 424),
 (('631633', '900207', '900222'), 408),
 (('100701130', '900102', '900212'), 414),
 (('100700853', '900102', '900212'), 566),
 (('900102', '900142', '900212'), 730)

In [29]:
#collecting value of rdds into lists
list1 = rr1.collect()
list2 = rr2.collect()
list3 = rr3.collect()


In [30]:
#collecting all 3_freq values of 3rdd and concat it into one_list
# we do it by defining a dictionary and here key is our 3_item_freq value and 
# value of dictionary is number of frequency thar for each rdd we add it to dictionary
dict_all = {}
for item in list1:
    key , val  = item[0] , item[1]
    if key in dict_all:
        dict_all[key] += val
    else:
        dict_all[key] = val

for item in list2:
    key , val  = item[0] , item[1]
    if key in dict_all:
        dict_all[key] += val
    else:
        dict_all[key] = val

for item in list3:
    key , val  = item[0] , item[1]
    if key in dict_all:
        dict_all[key] += val
    else:
        dict_all[key] = val
final_list = []
for key in dict_all:
    final_list.append((key , dict_all[key]))
sorted(final_list , key = lambda x : x[1] , reverse=True)

[(('900142', '900212', '900244'), 16388),
 (('631633', '900212', '900244'), 15187),
 (('100700853', '900212', '900244'), 10012),
 (('100700853', '900142', '900212'), 8577),
 (('631634', '900212', '900244'), 8419),
 (('900101', '900212', '900244'), 7372),
 (('100700839', '900212', '900244'), 7325),
 (('100700853', '900142', '900244'), 6937),
 (('631633', '631634', '900244'), 6261),
 (('631633', '900142', '900244'), 5594),
 (('900212', '900213', '900244'), 5579),
 (('100700841', '900101', '900236'), 5407),
 (('900102', '900212', '900244'), 5337),
 (('631633', '900142', '900212'), 5320),
 (('631633', '631634', '900212'), 5268),
 (('900225', '900268', '900269'), 5149),
 (('175', '900191', '900218'), 5038),
 (('900212', '900244', '900269'), 4646),
 (('900212', '900236', '900244'), 4534),
 (('900212', '900222', '900244'), 4331),
 (('631633', '631634', '900207'), 4299),
 (('900152', '900212', '900244'), 4142),
 (('631634', '900202', '900207'), 4066),
 (('100700841', '900212', '900244'), 4042)